In [1]:
import cv2
import matplotlib.pyplot as plt
from immas import MammogramImage , preprocessing , segmentation
from immas.io import read_dataset
from immas.basic_functions import show_image_plt
from immas import preprocessing

#Openkernel , Openkernel1 and Openkernel2 are tuples
def Segmentation(img,Openkernel= (10,10), Openkernel1= (13,13), OpenKernel2=(20,20), Kernel3=(20,20)):
    img = segmentation.multithresholding(img)
    img = segmentation.thresh_to_binary(img)
    #img1 = preprocessing.open(img,Openkernel)
    img1 = img
    img2 = preprocessing.open(img,Openkernel1)
    img3 = preprocessing.close(img2,Kernel3)
    return (img1,img2,img3)

#mean 1&2 are numbers
def Segmentation2(img, mean1=30,mean2=30, OpenKerne2=(20,20)):
    img = segmentation.mean_shift(img,mean1,mean2)
    img = segmentation.multithresholding(img)
    img = segmentation.thresh_to_binary(img)
    img = preprocessing.open(img,OpenKerne2)
    return img

from immas import preprocessing

def testPreProcessing (img, kernel_size = 20, clahe_kernel = 10.0):
    #img = preprocessing.resize(img, 0.5, 0.5)
    #img = preprocessing.close(img)
    #img = preprocessing.dilate(img)
    #img = preprocessing.clahe(img,10.0)
    img = preprocessing.open(img,(5,5))
    img = preprocessing.morphoEnhancement(img,kernel_size, clahe_kernel)
    img = preprocessing.waveletTransform(img)
    #img = preprocessing.erode(img,(3,3))    
    return img

In [2]:
#def Plot_Result(img, Morpho_kernel_size, Morpho_clahe_kernel, Kernel1, Kernel2, Kernel3):
def Plot_Result(img, Morpho_kernel_size=20, Morpho_clahe_kernel=10.0, Kernel1=(10,10), Kernel2=(23,23), Kernel3=(10,10)):    
    av_jaccard = 0
    m.read_data()
    m.image_data = testPreProcessing(m.image_data, Morpho_kernel_size, Morpho_clahe_kernel )

    #segment using both methods
    result1, result2, result4 = Segmentation(m.image_data, Kernel1, Kernel2, Kernel3)
    #result3 = Segmentation2(m.image_data, mean1,mean2, Openkernel2)

    #show both results
    plt.figure(figsize=(20, 40))
    plt.subplot(1,4,1)
    plt.axis("off")
    plt.imshow(m.cropped_ground_truth, interpolation="nearest", cmap=plt.cm.gray)
    plt.title('Groundtruth')

    plt.subplot(1,4,2)
    plt.axis("off")
    plt.imshow(result1, interpolation="nearest", cmap=plt.cm.gray)
    plt.title('Segmentation1')

    plt.subplot(1,4,3)
    plt.axis("off")
    plt.imshow(result2, interpolation="nearest", cmap=plt.cm.gray)   
    plt.title('Segmentation2')

    plt.subplot(1,4,4)
    plt.axis("off")
    plt.imshow(result4, interpolation="nearest", cmap=plt.cm.gray)          
    plt.title('Segmentation3')
    av_jaccard = segmentation.jaccard_index(result4,  m.cropped_ground_truth)
    print(av_jaccard)
    plt.show()
    return av_jaccard

In [3]:
def main():
    import multiprocessing as mp

    data_set = read_dataset(image_folder="../dataset/masses_examples",
                mask_folder="../dataset/masks",
                results_folder="../dataset/groundtruth",
                train_set_fraction=1)

    print("Number of images for training is {0}, number of images for testing is {1}".format(
        len(data_set["train"]), len(data_set["test"])))

    img = data_set["train"]

    pool = mp.Pool(processes=4)
    results = [pool.apply(Plot_Result,(m)) for m in img]

#Plot_Result(img, Morpho_kernel_size=20, Morpho_clahe_kernel=10.0, Kernel1=(20,20), Kernel2=(27,27), Kernel3=(5,5))

main()

In [ ]:
main()

Reading list of files...
Reading mamograms images and all additional data...
All data have been successfully loaded.
Number of images for training is 107, number of images for testing is 0


In [ ]:
print("Number of images for training is {0}, number of images for testing is {1}".format(
    len(data_set["train"]), len(data_set["test"])))

av_jaccard = 0
img = data_set["train"]
count = 0
for m in img:
    m.read_data()
    m.image_data = testPreProcessing(m.image_data,20,10.0)
    #Best parameters should be made default ... for segmentations
    Seg, SegA, SegB = Segmentation(m.image_data, Openkernel=(20,20), Openkernel1=(23,23), OpenKernel2=(20,20), Kernel3=(5,10))
    av_jaccard = av_jaccard + segmentation.jaccard_index(SegB,  m.cropped_ground_truth)
    count = count + 1
    print(segmentation.jaccard_index(SegB,  m.cropped_ground_truth))

print("Average jaccard index for training set (Segmentaion1)is {}".format(av_jaccard/count))
#print("Average jaccard index for training set (Segmentaion2)is {}".format(av_jaccard2/count))

In [ ]:
path_image = "../dataset/images/22579730_bbd6a3a35438c11b_MG_R_ML_ANON.tif"
path_mask = "../dataset/masks/20587080_b6a4f750c6df4f90_MG_R_ML_ANON.png"
pectoral_muscle = "../dataset/pectoral_muscle_masks/20587080_b6a4f750c6df4f90_MG_R_ML_ANON.tif"
mm = MammogramImage(path_image, path_mask, pmuscle_mask_path=pectoral_muscle)


img = mm.image_data


enhanced = preprocessing.clahe(img)
morpho = preprocessing.morphoEnhancement(img)
img_wavelet = preprocessing.waveletTransform(morpho)

plt.figure(figsize=(20, 10))
plt.subplot(1,4,1)
plt.axis("off")
plt.imshow(img, interpolation="nearest", cmap=plt.cm.gray)
plt.title('Image')

plt.subplot(1,4,2)
plt.axis("off")
plt.imshow(enhanced, interpolation="nearest", cmap=plt.cm.gray)
plt.title('CLAHE')

plt.subplot(1,4,3)
plt.axis("off")
plt.imshow(morpho, interpolation="nearest", cmap=plt.cm.gray)
plt.title('Morpho')

plt.subplot(1,4,4)
plt.axis("off")
plt.imshow(img_wavelet, interpolation="nearest", cmap=plt.cm.gray)
plt.title('Wavelet and Morpho')
plt.show()